# **Introduction:**

This file serves to compare the inference systems that had been developed.

**Date Created: 16/2/2025**

**Date Modified: 18/2/2025**

# **Import Packages:**

This section imports the necessary packages.

In [18]:
# import packages:
import numpy as np
import time
import os
from keras.models import load_model
import tensorflow as tf
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler
from pickle import load
from ANFIS_Custom_Layers import *
from PythonFISFunctionV3 import *

# **Pre-amble:**

This section defines paths and variables to be used in the model loading.

In [23]:
# define the ANN directory path:
ann_path = os.path.join(os.getcwd(), 'ANN_Model')

# define the ANFIS directory path:
anfis_path = os.path.join(os.getcwd(), 'ANFIS_Model')

# define the dictionary of custom objects for the ANFIS:
custom_objects = {
    # # layers:
    'MF_Layer'          : MF_Layer,
    'FS_Layer'          : FS_Layer,
    'NM_Layer'          : NM_Layer,
    'CN_Layer'          : CN_Layer,
    'O_Layer'           : O_Layer,

    # other:
    'OrderedConstraint' : OrderedConstraint(),
    'mse'               : MeanSquaredError()
}

# define the batch size for model warming:
batch_size = 1

# **Create FIS:**

This section creates the rule-base for the FIS.

In [24]:
rulebase = fis_create()

# **Load the ANN Model:**

This section loads the ANN model and the scaler that is used for the ANN.

In [25]:
# load the ann model:
ann_model = load_model(ann_path + '/ann_model.h5', custom_objects = {'mse' : MeanSquaredError()})

# load the ann scaler:
ann_scaler = load(open(ann_path + '/ann_scaler.pkl', 'rb'))

# need to warm the model to prevent retracing of computational graph on first inference:
dummy_input = np.zeros((batch_size, 3), dtype = np.float32)
dummy_input = ann_scaler.transform(dummy_input)
ann_model.predict(dummy_input)

c:\Users\mtidd2\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


array([[9.33442]], dtype=float32)

# **Load the ANFIS Model:**

This section loads the ANFIS model and the scaler.

In [26]:
# load the model:
anfis_model = load_model(anfis_path + '/anfis_model.h5', custom_objects = custom_objects)
anfis_model.compile(optimizer="adam", loss="mse") 

# load the scaler:
anfis_scaler = load(open(anfis_path + '/anfis_scaler.pkl', 'rb'))

# warmup the model:
dummy_input = np.zeros((batch_size, 3), dtype=np.float32)  
dummy_input = anfis_scaler.transform(dummy_input)
anfis_model.predict(dummy_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[9.028122]], dtype=float32)

# **Generate Testing Data:**

This section generates the data that the models are tested on. This is done by sampling the universe of discourse of each variable and predicting the suitability using the FIS.

In [58]:
# initialize testing data:
test_data = []

# generate data:
for i in range(100):
    lh = float(np.random.randint(0, 10 + 1))
    dtt = np.random.uniform(0, 25)
    dh = np.random.uniform(0, 50)

    suit = fis_solve(rulebase, lh, dtt, dh)
    test_data.append([lh, dtt, dh, suit])

# convert to numpy array:
test_data = np.array(test_data)

# **Test the Models:**

This section tests the models against one another on the same input dataset

In [ ]:
# testing the ANN: